<a href="https://colab.research.google.com/github/senasung37/NLP/blob/main/count-based/lemmatization_tfidf_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#웹스크레이핑

In [ ]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import numpy as np
import nltk

In [ ]:
#네이버 영화 웹 스크레이핑 함수들
BASE_URL = "https://movie.naver.com/movie"

def get_page(page_url):

    page = requests.get(page_url)
    soup = BeautifulSoup(page.content, 'html.parser')

    return soup, page

# 평균점수를 구해주는 함수
def get_avg_stars(reviews):
   
    review_star = []
    for r in reviews:
        review_star.append(r['review_star'])

    avg = sum(review_star) / len(review_star)

    return avg

# url에서 가지고 있는 영화 제목의 코드 번호를 리턴해주는 함수
def get_movie_code(movie_title):
    search_url = f"{BASE_URL}/search/result.naver?query={movie_title}&section=all&ie=utf8"
    soup = get_page(search_url)[0]

    movie_code = int(soup.find(class_='result_thumb').a.get('href').split('=')[1])

    return movie_code

# 영화코드와 구하고자 하는 페이지 수를 입력하면 리뷰와 평점을 리스트로 반환하는 함수  
def get_reviews(movie_code, page_num=1):
    review_list = []

    # 입력된 페이지 수만큼 작업을 진행함. 
    for i in range(1,page_num+1):
        review_url = f"{BASE_URL}/point/af/list.naver?st=mcode&sword={movie_code}&target=after&page={page_num}" 
        soup = get_page(review_url)[0]
        comment = soup.find_all(class_='title')
        
        # class = title에 포함된 정보들을 모두 찾아서 comment라는 변수에 저장
        for report in comment:
            cache={}
            cache['review_text'] = report.contents[6].strip() # 불러온 자료를 리스트로 변경한후 6번째에 있는 리뷰를 반환 (불필요한 뛰어쓰기 삭제)
            cache['review_star'] = int(report.contents[3].em.text) # 불러온 자료를 리스트로 변경한 후 3번째에 있는 평점을 반환.

            review_list.append(cache)
    return review_list

def scrape_by_review_num(movie_title, review_num):

    code = get_movie_code(movie_title)

    # get_reviews 함수를 활용하여 리뷰들을 불러온다.
    init_list = get_reviews(code,round(review_num/10)+1)
    
    # 리뷰들을 몇개까지 불러올지에 따라 리턴해 준다.
    reviews = init_list[0:review_num]

    return reviews

def scrape_by_page_num(movie_title, page_num=10):

    code = get_movie_code(movie_title)

    # get_reviews 함수를 활용하여 영화 제목과 페이지 수를 넘기면 리스트를 리턴해 준다.
    reviews = get_reviews(code, page_num)

    return reviews

In [ ]:
#영화 리뷰 불러오기
movie_1 = scrape_by_review_num("더 배트맨", 1000)
movie_2 = scrape_by_review_num("스파이더맨: 노 웨이 홈", 1000)
movie_3 = scrape_by_review_num("해적: 도깨비 깃발", 1000)

In [ ]:
df_movie_1 = pd.DataFrame.from_dict(movie_1)
df_movie_1['movie'] = "batman"

df_movie_2 = pd.DataFrame.from_dict(movie_2)
df_movie_2['movie'] = "spiderman"

df_movie_3 = pd.DataFrame.from_dict(movie_3)
df_movie_3['movie'] = "pirate"


In [ ]:
df_movie = pd.concat([df_movie_1, df_movie_2, df_movie_3])
df = df_movie.drop_duplicates()
df

,review_text,review_star,movie
0,"복수에서 메시아로, 그림자에서 어둠 속의 빛으로",8,batman
1,베트맨 시리즈는 처음이지만 로버트 패튼슨 버전의 베트맨 극호! 정보없이 봤는데 로버...,10,batman
2,시간떼우려고 갔다 자고 왔네여 배우가 계속 혼잣말로 속삭여서 너무 졸리게 만든다 ...,2,batman
3,bad bad bad bad,2,batman
4,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,10,batman
5,이정도로 잘뽑아낼줄은 몰랐는데 놀랍네,7,batman
6,전체적으로 스토리 짜임새가 괜찮았던 거 같습니다.로버트 패틴슨 연기도 좋았구요. 브...,9,batman
7,재미 ㅈ나게 없더라. 조커 급 기대했다 하품만하다 옴,2,batman
8,"뻔하다 못 해 진부한 스토리, 변비 걸린 것 같은 전개",4,batman
9,호들갑은 떠는데 현실은 100만도 안 됨,3,batman


In [ ]:
#데이터 셔플 및 NA 삭제
from sklearn.utils import shuffle
df['review_text'].replace(' ', np.nan, inplace=True)
df = df.dropna(axis=0, how='any')
df = shuffle(df_movie).reset_index(drop=True)
df

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


,review_text,review_star,movie
0,영화보면서 댓글남기는중... 최근 10년 동안본영화중 역대급으로 재미없음... 스토...,1,pirate
1,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,10,batman
2,스토리 전개가 쥑입니다요..^^,10,spiderman
3,bad bad bad bad,2,batman
4,와.. 정말 연기가.. 이걸 어떻게 본담 ㅎㅎㅎ 손예진 김남길 흉내내는 삼류..,1,pirate
...,...,...,...
2995,영화내내 다섯살짜리 애기같은 스파이더맨에 끝까지 뺑이만 치는 닥터 스트레인지모든 스...,1,spiderman
2996,소니의 스파이더맨들과의 콜라보가 될 줄이야...,8,spiderman
2997,bad bad bad bad,2,batman
2998,bad bad bad bad,2,batman


In [ ]:
df.shape

(3000, 3)

In [ ]:
df.groupby(df['review_text'].tolist(),as_index=False).size()

,index,size
0,,100
1,10분보고 껐네요.평점보러왔더니 다들 나랑 같으시네,100
2,bad bad bad bad,100
3,다음 영화가 너무 기대되네요!,100
4,베트맨 시리즈는 처음이지만 로버트 패튼슨 버전의 베트맨 극호! 정보없이 봤는데 로버...,100
5,"복수에서 메시아로, 그림자에서 어둠 속의 빛으로",100
6,"뻔하다 못 해 진부한 스토리, 변비 걸린 것 같은 전개",100
7,소니의 스파이더맨들과의 콜라보가 될 줄이야...,100
8,스토리 전개가 쥑입니다요..^^,100
9,스파이더맨 시리즈에 대한 선물같은 영화. 감동적인 순간들이 많아요. 윌렘 대포 최고,100


In [ ]:
df.head(50)

,review_text,review_star,movie
0,영화보면서 댓글남기는중... 최근 10년 동안본영화중 역대급으로 재미없음... 스토...,1,pirate
1,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,10,batman
2,스토리 전개가 쥑입니다요..^^,10,spiderman
3,bad bad bad bad,2,batman
4,와.. 정말 연기가.. 이걸 어떻게 본담 ㅎㅎㅎ 손예진 김남길 흉내내는 삼류..,1,pirate
5,시간떼우려고 갔다 자고 왔네여 배우가 계속 혼잣말로 속삭여서 너무 졸리게 만든다 ...,2,batman
6,시간떼우려고 갔다 자고 왔네여 배우가 계속 혼잣말로 속삭여서 너무 졸리게 만든다 ...,2,batman
7,왜.. 대체 왜...한효주와 강하늘을 같이 썼을까..한효주씨는 캐릭터 한계가 있는데...,2,pirate
8,전체적으로 스토리 짜임새가 괜찮았던 거 같습니다.로버트 패틴슨 연기도 좋았구요. 브...,9,batman
9,한효주배우님 연기 못봐주겠어요20분보다 끝,2,pirate


#NLP

##전처리

In [ ]:
import re 
def clean_text(texts):
  #remove punctuation 
  review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts))
  review = re.sub(r'[ㅜ\ㅜㅜ\..\♡\ㅠ\ㅠㅠ]', ' ',str(texts)) 

  #remove number
  #review = re.sub(r'\d+','', str(review))
  #lower case 
  #review = review.lower() 

  #remove extra space
  review = re.sub(r'\s+', ' ', review) 
  #remove Html tags 
  review = re.sub(r'<[^>]+>','',review) 
  #remove spaces
  review = re.sub(r'\s+', ' ', review) 
  #remove space from start
  review = re.sub(r"^\s+", '', review) 
  #remove space from the end  
  review = re.sub(r'\s+$', '', review) 
  return review

#출처: https://ebbnflow.tistory.com/246 [Dev Log : 삶은 확률의 구름]

In [ ]:
def tokenizer(text):
  text = clean_text(text)
  text = text.split()
  return text
  
  
df['clean_text'] = df["review_text"].apply(clean_text)
df['tokens'] = df['review_text'].apply(tokenizer)
df

,review_text,review_star,movie,clean_text,tokens
0,영화보면서 댓글남기는중... 최근 10년 동안본영화중 역대급으로 재미없음... 스토...,1,pirate,영화보면서 댓글남기는중 최근 10년 동안본영화중 역대급으로 재미없음 스토리 연기 특...,"[영화보면서, 댓글남기는중, 최근, 10년, 동안본영화중, 역대급으로, 재미없음, ..."
1,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,10,batman,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,"[요즘, 정치시즌에도, 뭔가, 잘맞는낌, 강추합니다]"
2,스토리 전개가 쥑입니다요..^^,10,spiderman,스토리 전개가 쥑입니다요 ^^,"[스토리, 전개가, 쥑입니다요, ^^]"
3,bad bad bad bad,2,batman,bad bad bad bad,"[bad, bad, bad, bad]"
4,와.. 정말 연기가.. 이걸 어떻게 본담 ㅎㅎㅎ 손예진 김남길 흉내내는 삼류..,1,pirate,와 정말 연기가 이걸 어떻게 본담 ㅎㅎㅎ 손예진 김남길 흉내내는 삼류,"[와, 정말, 연기가, 이걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉내내는,..."
...,...,...,...,...,...
2995,영화내내 다섯살짜리 애기같은 스파이더맨에 끝까지 뺑이만 치는 닥터 스트레인지모든 스...,1,spiderman,영화내내 다섯살짜리 애기같은 스파이더맨에 끝까지 뺑이만 치는 닥터 스트레인지모든 스...,"[영화내내, 다섯살짜리, 애기같은, 스파이더맨에, 끝까지, 뺑이만, 치는, 닥터, ..."
2996,소니의 스파이더맨들과의 콜라보가 될 줄이야...,8,spiderman,소니의 스파이더맨들과의 콜라보가 될 줄이야,"[소니의, 스파이더맨들과의, 콜라보가, 될, 줄이야]"
2997,bad bad bad bad,2,batman,bad bad bad bad,"[bad, bad, bad, bad]"
2998,bad bad bad bad,2,batman,bad bad bad bad,"[bad, bad, bad, bad]"


In [ ]:
#Top 10 토큰
from collections import Counter

word_counts = Counter()
df['tokens'].apply(lambda x: word_counts.update(x))
word_counts.most_common(30)

[('연기', 500),
 ('스파이더맨', 500),
 ('bad', 400),
 ('너무', 400),
 ('가장', 400),
 ('스토리', 300),
 ('왜', 300),
 ('잘', 300),
 ('로버트', 300),
 ('전', 300),
 ('정말', 200),
 ('만든다', 200),
 ('거', 200),
 ('안', 200),
 ('베트맨', 200),
 ('패튼슨', 200),
 ('또', 200),
 ('영화가', 200),
 ('스토리,', 200),
 ('감동적인', 200),
 ('사람은', 200),
 ('영화', 200),
 ('최고', 200),
 ('영화보면서', 100),
 ('댓글남기는중', 100),
 ('최근', 100),
 ('10년', 100),
 ('동안본영화중', 100),
 ('역대급으로', 100),
 ('재미없음', 100)]

In [ ]:
#토크나이저 생성
import spacy
from spacy.tokenizer import Tokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 

#출처: https://mr-doosun.tistory.com/24 [고졸 입니다만..]

nlp = spacy.load("en_core_web_sm")
tokenizer = Tokenizer(nlp.vocab)
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#불용어 지정 
stopwords = ['이', '그', '근데', '더', '잘', '같은', '수', '등', '영화', '그래도', '그래서', '어쩌면', '왜나면', '아니면', '좀', '사실', '이런', '저런', '아', '진짜', '없는', '와', '몇', '너무', '같이', '음']

In [ ]:
tokens = []

for doc in tokenizer.pipe(df['clean_text']):    
    doc_tokens = []

    for token in doc:
        # 토큰이 불용어와 구두점이 아니면 저장
        if (token.is_stop == False) & (token.is_punct == False) & (token.text.lower() not in stopwords):
            doc_tokens.append(token.text.lower())
   
    tokens.append(doc_tokens)
    
df['tokens_wo_sw'] = tokens
df.head(20)

,review_text,review_star,movie,clean_text,tokens,tokens_wo_sw
0,영화보면서 댓글남기는중... 최근 10년 동안본영화중 역대급으로 재미없음... 스토...,1,pirate,영화보면서 댓글남기는중 최근 10년 동안본영화중 역대급으로 재미없음 스토리 연기 특...,"[영화보면서, 댓글남기는중, 최근, 10년, 동안본영화중, 역대급으로, 재미없음, ...","[영화보면서, 댓글남기는중, 최근, 10년, 동안본영화중, 역대급으로, 재미없음, ..."
1,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,10,batman,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,"[요즘, 정치시즌에도, 뭔가, 잘맞는낌, 강추합니다]","[요즘, 정치시즌에도, 뭔가, 잘맞는낌, 강추합니다]"
2,스토리 전개가 쥑입니다요..^^,10,spiderman,스토리 전개가 쥑입니다요 ^^,"[스토리, 전개가, 쥑입니다요, ^^]","[스토리, 전개가, 쥑입니다요, ^^]"
3,bad bad bad bad,2,batman,bad bad bad bad,"[bad, bad, bad, bad]","[bad, bad, bad, bad]"
4,와.. 정말 연기가.. 이걸 어떻게 본담 ㅎㅎㅎ 손예진 김남길 흉내내는 삼류..,1,pirate,와 정말 연기가 이걸 어떻게 본담 ㅎㅎㅎ 손예진 김남길 흉내내는 삼류,"[와, 정말, 연기가, 이걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉내내는,...","[정말, 연기가, 이걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉내내는, 삼류]"
5,시간떼우려고 갔다 자고 왔네여 배우가 계속 혼잣말로 속삭여서 너무 졸리게 만든다 ...,2,batman,시간떼우려고 갔다 자고 왔네여 배우가 계속 혼잣말로 속삭여서 너무 졸리게 만든다 보...,"[시간떼우려고, 갔다, 자고, 왔네여, 배우가, 계속, 혼잣말로, 속삭여서, 너무,...","[시간떼우려고, 갔다, 자고, 왔네여, 배우가, 계속, 혼잣말로, 속삭여서, 졸리게..."
6,시간떼우려고 갔다 자고 왔네여 배우가 계속 혼잣말로 속삭여서 너무 졸리게 만든다 ...,2,batman,시간떼우려고 갔다 자고 왔네여 배우가 계속 혼잣말로 속삭여서 너무 졸리게 만든다 보...,"[시간떼우려고, 갔다, 자고, 왔네여, 배우가, 계속, 혼잣말로, 속삭여서, 너무,...","[시간떼우려고, 갔다, 자고, 왔네여, 배우가, 계속, 혼잣말로, 속삭여서, 졸리게..."
7,왜.. 대체 왜...한효주와 강하늘을 같이 썼을까..한효주씨는 캐릭터 한계가 있는데...,2,pirate,왜 대체 왜 한효주와 강하늘을 같이 썼을까 한효주씨는 캐릭터 한계가 있는데 이 배역...,"[왜, 대체, 왜, 한효주와, 강하늘을, 같이, 썼을까, 한효주씨는, 캐릭터, 한계...","[왜, 대체, 왜, 한효주와, 강하늘을, 썼을까, 한효주씨는, 캐릭터, 한계가, 있..."
8,전체적으로 스토리 짜임새가 괜찮았던 거 같습니다.로버트 패틴슨 연기도 좋았구요. 브...,9,batman,전체적으로 스토리 짜임새가 괜찮았던 거 같습니다 로버트 패틴슨 연기도 좋았구요 브루...,"[전체적으로, 스토리, 짜임새가, 괜찮았던, 거, 같습니다, 로버트, 패틴슨, 연기...","[전체적으로, 스토리, 짜임새가, 괜찮았던, 거, 같습니다, 로버트, 패틴슨, 연기..."
9,한효주배우님 연기 못봐주겠어요20분보다 끝,2,pirate,한효주배우님 연기 못봐주겠어요20분보다 끝,"[한효주배우님, 연기, 못봐주겠어요20분보다, 끝]","[한효주배우님, 연기, 못봐주겠어요20분보다, 끝]"


In [ ]:
#불용어 제거 후 top10 words
df['tokens_wo_sw'].apply(lambda x: word_counts.update(x))
word_counts.most_common(10)

[('연기', 1000),
 ('스파이더맨', 1000),
 ('bad', 800),
 ('가장', 800),
 ('스토리', 600),
 ('왜', 600),
 ('로버트', 600),
 ('전', 600),
 ('정말', 400),
 ('너무', 400)]

#Lemmatization

##soylemma

In [ ]:
!pip install soylemma

     |████████████████████████████████| 124 kB 5.3 MB/s 


In [ ]:
from soylemma import Lemmatizer

lemmatizer = Lemmatizer()
lemmatizer.analyze('차가우니까')

[(('차갑', 'Adjective'), ('으니까', 'Eomi'))]

In [ ]:
lemmatizer.lemmatize('차가우니까')

[('차갑다', 'Adjective')]

##konlpy

In [ ]:
#라이브러리 설치
!pip install konlpy 
import konlpy
from konlpy.tag import Kkma
from konlpy.utils import pprint
from konlpy.tag import Okt

     |████████████████████████████████| 19.4 MB 172 kB/s 
     |████████████████████████████████| 448 kB 62.5 MB/s 


In [ ]:
#출처: https://dacon.io/codeshare/1808
kkma = Kkma()

In [ ]:
#kkma 형태소 단위로 문장분리
df['lem_kkma_morph'] = df['clean_text'].apply(kkma.morphs)
df.head(5)

,review_text,review_star,movie,clean_text,tokens,tokens_wo_sw,lem_kkma_morph
0,영화보면서 댓글남기는중... 최근 10년 동안본영화중 역대급으로 재미없음... 스토...,1,pirate,영화보면서 댓글남기는중 최근 10년 동안본영화중 역대급으로 재미없음 스토리 연기 특...,"[영화보면서, 댓글남기는중, 최근, 10년, 동안본영화중, 역대급으로, 재미없음, ...","[영화보면서, 댓글남기는중, 최근, 10년, 동안본영화중, 역대급으로, 재미없음, ...","[영화, 보, 면서, 댓, 글, 남기, 는, 중, 최근, 10, 년, 동안, 본, ..."
1,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,10,batman,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,"[요즘, 정치시즌에도, 뭔가, 잘맞는낌, 강추합니다]","[요즘, 정치시즌에도, 뭔가, 잘맞는낌, 강추합니다]","[요즘, 정치, 시즌, 에, 도, 뭔, 가, 잘, 맞, 는, 끼, ㅁ, 강추, 하,..."
2,스토리 전개가 쥑입니다요..^^,10,spiderman,스토리 전개가 쥑입니다요 ^^,"[스토리, 전개가, 쥑입니다요, ^^]","[스토리, 전개가, 쥑입니다요, ^^]","[스토리, 전개, 가, 쥑이, ㅂ니다요, ^^]"
3,bad bad bad bad,2,batman,bad bad bad bad,"[bad, bad, bad, bad]","[bad, bad, bad, bad]","[bad, bad, bad, bad]"
4,와.. 정말 연기가.. 이걸 어떻게 본담 ㅎㅎㅎ 손예진 김남길 흉내내는 삼류..,1,pirate,와 정말 연기가 이걸 어떻게 본담 ㅎㅎㅎ 손예진 김남길 흉내내는 삼류,"[와, 정말, 연기가, 이걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉내내는,...","[정말, 연기가, 이걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉내내는, 삼류]","[오, 아, 정말, 연기, 가, 이것, 을, 어떻, 게, 보, ㄴ담, ㅎㅎㅎ, 손,..."


In [ ]:
#kkma 명사 추출
df['lem_kkma_noun'] = df['clean_text'].apply(kkma.nouns)
df.head(5)

,review_text,review_star,movie,clean_text,tokens,tokens_wo_sw,lem_kkma_morph,lem_kkma_noun
0,영화보면서 댓글남기는중... 최근 10년 동안본영화중 역대급으로 재미없음... 스토...,1,pirate,영화보면서 댓글남기는중 최근 10년 동안본영화중 역대급으로 재미없음 스토리 연기 특...,"[영화보면서, 댓글남기는중, 최근, 10년, 동안본영화중, 역대급으로, 재미없음, ...","[영화보면서, 댓글남기는중, 최근, 10년, 동안본영화중, 역대급으로, 재미없음, ...","[영화, 보, 면서, 댓, 글, 남기, 는, 중, 최근, 10, 년, 동안, 본, ...","[영화, 댓, 댓글, 글, 중, 최근, 10, 10년, 년, 동안, 동안본영화중, ..."
1,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,10,batman,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,"[요즘, 정치시즌에도, 뭔가, 잘맞는낌, 강추합니다]","[요즘, 정치시즌에도, 뭔가, 잘맞는낌, 강추합니다]","[요즘, 정치, 시즌, 에, 도, 뭔, 가, 잘, 맞, 는, 끼, ㅁ, 강추, 하,...","[요즘, 정치, 정치시즌, 시즌, 강추]"
2,스토리 전개가 쥑입니다요..^^,10,spiderman,스토리 전개가 쥑입니다요 ^^,"[스토리, 전개가, 쥑입니다요, ^^]","[스토리, 전개가, 쥑입니다요, ^^]","[스토리, 전개, 가, 쥑이, ㅂ니다요, ^^]","[스토리, 전개]"
3,bad bad bad bad,2,batman,bad bad bad bad,"[bad, bad, bad, bad]","[bad, bad, bad, bad]","[bad, bad, bad, bad]",[]
4,와.. 정말 연기가.. 이걸 어떻게 본담 ㅎㅎㅎ 손예진 김남길 흉내내는 삼류..,1,pirate,와 정말 연기가 이걸 어떻게 본담 ㅎㅎㅎ 손예진 김남길 흉내내는 삼류,"[와, 정말, 연기가, 이걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉내내는,...","[정말, 연기가, 이걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉내내는, 삼류]","[오, 아, 정말, 연기, 가, 이것, 을, 어떻, 게, 보, ㄴ담, ㅎㅎㅎ, 손,...","[연기, 이것, 손, 손예진, 예진, 김, 삼류]"


In [ ]:
#OKT
Okt = Okt()

In [ ]:
df['lem_okt_morph'] = df['clean_text'].apply(Okt.morphs)
df.head(5)

,review_text,review_star,movie,clean_text,tokens,tokens_wo_sw,lem_kkma_morph,lem_kkma_noun,lem_okt_morph
0,영화보면서 댓글남기는중... 최근 10년 동안본영화중 역대급으로 재미없음... 스토...,1,pirate,영화보면서 댓글남기는중 최근 10년 동안본영화중 역대급으로 재미없음 스토리 연기 특...,"[영화보면서, 댓글남기는중, 최근, 10년, 동안본영화중, 역대급으로, 재미없음, ...","[영화보면서, 댓글남기는중, 최근, 10년, 동안본영화중, 역대급으로, 재미없음, ...","[영화, 보, 면서, 댓, 글, 남기, 는, 중, 최근, 10, 년, 동안, 본, ...","[영화, 댓, 댓글, 글, 중, 최근, 10, 10년, 년, 동안, 동안본영화중, ...","[영화, 보면서, 댓글, 남기는, 중, 최근, 10년, 동안, 본, 영화, 중, 역..."
1,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,10,batman,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,"[요즘, 정치시즌에도, 뭔가, 잘맞는낌, 강추합니다]","[요즘, 정치시즌에도, 뭔가, 잘맞는낌, 강추합니다]","[요즘, 정치, 시즌, 에, 도, 뭔, 가, 잘, 맞, 는, 끼, ㅁ, 강추, 하,...","[요즘, 정치, 정치시즌, 시즌, 강추]","[요즘, 정치, 시즌, 에도, 뭔가, 잘맞는, 낌, 강, 추합니다]"
2,스토리 전개가 쥑입니다요..^^,10,spiderman,스토리 전개가 쥑입니다요 ^^,"[스토리, 전개가, 쥑입니다요, ^^]","[스토리, 전개가, 쥑입니다요, ^^]","[스토리, 전개, 가, 쥑이, ㅂ니다요, ^^]","[스토리, 전개]","[스토리, 전개, 가, 쥑, 입니다요, ^^]"
3,bad bad bad bad,2,batman,bad bad bad bad,"[bad, bad, bad, bad]","[bad, bad, bad, bad]","[bad, bad, bad, bad]",[],"[bad, bad, bad, bad]"
4,와.. 정말 연기가.. 이걸 어떻게 본담 ㅎㅎㅎ 손예진 김남길 흉내내는 삼류..,1,pirate,와 정말 연기가 이걸 어떻게 본담 ㅎㅎㅎ 손예진 김남길 흉내내는 삼류,"[와, 정말, 연기가, 이걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉내내는,...","[정말, 연기가, 이걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉내내는, 삼류]","[오, 아, 정말, 연기, 가, 이것, 을, 어떻, 게, 보, ㄴ담, ㅎㅎㅎ, 손,...","[연기, 이것, 손, 손예진, 예진, 김, 삼류]","[와, 정말, 연기, 가, 이, 걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉..."


In [ ]:
#단어 추출
df['lem_okt_noun'] = df['clean_text'].apply(Okt.nouns)
df.head(5)

,review_text,review_star,movie,clean_text,tokens,tokens_wo_sw,lem_kkma_morph,lem_kkma_noun,lem_okt_morph,lem_okt_noun
0,영화보면서 댓글남기는중... 최근 10년 동안본영화중 역대급으로 재미없음... 스토...,1,pirate,영화보면서 댓글남기는중 최근 10년 동안본영화중 역대급으로 재미없음 스토리 연기 특...,"[영화보면서, 댓글남기는중, 최근, 10년, 동안본영화중, 역대급으로, 재미없음, ...","[영화보면서, 댓글남기는중, 최근, 10년, 동안본영화중, 역대급으로, 재미없음, ...","[영화, 보, 면서, 댓, 글, 남기, 는, 중, 최근, 10, 년, 동안, 본, ...","[영화, 댓, 댓글, 글, 중, 최근, 10, 10년, 년, 동안, 동안본영화중, ...","[영화, 보면서, 댓글, 남기는, 중, 최근, 10년, 동안, 본, 영화, 중, 역...","[영화, 댓글, 중, 최근, 동안, 영화, 역대, 스토리, 연기, 특수, 효과, 등..."
1,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,10,batman,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,"[요즘, 정치시즌에도, 뭔가, 잘맞는낌, 강추합니다]","[요즘, 정치시즌에도, 뭔가, 잘맞는낌, 강추합니다]","[요즘, 정치, 시즌, 에, 도, 뭔, 가, 잘, 맞, 는, 끼, ㅁ, 강추, 하,...","[요즘, 정치, 정치시즌, 시즌, 강추]","[요즘, 정치, 시즌, 에도, 뭔가, 잘맞는, 낌, 강, 추합니다]","[요즘, 정치, 시즌, 뭔가, 낌, 강]"
2,스토리 전개가 쥑입니다요..^^,10,spiderman,스토리 전개가 쥑입니다요 ^^,"[스토리, 전개가, 쥑입니다요, ^^]","[스토리, 전개가, 쥑입니다요, ^^]","[스토리, 전개, 가, 쥑이, ㅂ니다요, ^^]","[스토리, 전개]","[스토리, 전개, 가, 쥑, 입니다요, ^^]","[스토리, 전개, 쥑]"
3,bad bad bad bad,2,batman,bad bad bad bad,"[bad, bad, bad, bad]","[bad, bad, bad, bad]","[bad, bad, bad, bad]",[],"[bad, bad, bad, bad]",[]
4,와.. 정말 연기가.. 이걸 어떻게 본담 ㅎㅎㅎ 손예진 김남길 흉내내는 삼류..,1,pirate,와 정말 연기가 이걸 어떻게 본담 ㅎㅎㅎ 손예진 김남길 흉내내는 삼류,"[와, 정말, 연기가, 이걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉내내는,...","[정말, 연기가, 이걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉내내는, 삼류]","[오, 아, 정말, 연기, 가, 이것, 을, 어떻, 게, 보, ㄴ담, ㅎㅎㅎ, 손,...","[연기, 이것, 손, 손예진, 예진, 김, 삼류]","[와, 정말, 연기, 가, 이, 걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉...","[정말, 연기, 걸, 손예진, 김남길, 흉내내, 삼류]"


In [ ]:
df

,review_text,review_star,movie,clean_text,tokens,tokens_wo_sw,lem_kkma_morph,lem_kkma_noun,lem_okt_morph,lem_okt_noun
0,영화보면서 댓글남기는중... 최근 10년 동안본영화중 역대급으로 재미없음... 스토...,1,pirate,영화보면서 댓글남기는중 최근 10년 동안본영화중 역대급으로 재미없음 스토리 연기 특...,"[영화보면서, 댓글남기는중, 최근, 10년, 동안본영화중, 역대급으로, 재미없음, ...","[영화보면서, 댓글남기는중, 최근, 10년, 동안본영화중, 역대급으로, 재미없음, ...","[영화, 보, 면서, 댓, 글, 남기, 는, 중, 최근, 10, 년, 동안, 본, ...","[영화, 댓, 댓글, 글, 중, 최근, 10, 10년, 년, 동안, 동안본영화중, ...","[영화, 보면서, 댓글, 남기는, 중, 최근, 10년, 동안, 본, 영화, 중, 역...","[영화, 댓글, 중, 최근, 동안, 영화, 역대, 스토리, 연기, 특수, 효과, 등..."
1,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,10,batman,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,"[요즘, 정치시즌에도, 뭔가, 잘맞는낌, 강추합니다]","[요즘, 정치시즌에도, 뭔가, 잘맞는낌, 강추합니다]","[요즘, 정치, 시즌, 에, 도, 뭔, 가, 잘, 맞, 는, 끼, ㅁ, 강추, 하,...","[요즘, 정치, 정치시즌, 시즌, 강추]","[요즘, 정치, 시즌, 에도, 뭔가, 잘맞는, 낌, 강, 추합니다]","[요즘, 정치, 시즌, 뭔가, 낌, 강]"
2,스토리 전개가 쥑입니다요..^^,10,spiderman,스토리 전개가 쥑입니다요 ^^,"[스토리, 전개가, 쥑입니다요, ^^]","[스토리, 전개가, 쥑입니다요, ^^]","[스토리, 전개, 가, 쥑이, ㅂ니다요, ^^]","[스토리, 전개]","[스토리, 전개, 가, 쥑, 입니다요, ^^]","[스토리, 전개, 쥑]"
3,bad bad bad bad,2,batman,bad bad bad bad,"[bad, bad, bad, bad]","[bad, bad, bad, bad]","[bad, bad, bad, bad]",[],"[bad, bad, bad, bad]",[]
4,와.. 정말 연기가.. 이걸 어떻게 본담 ㅎㅎㅎ 손예진 김남길 흉내내는 삼류..,1,pirate,와 정말 연기가 이걸 어떻게 본담 ㅎㅎㅎ 손예진 김남길 흉내내는 삼류,"[와, 정말, 연기가, 이걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉내내는,...","[정말, 연기가, 이걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉내내는, 삼류]","[오, 아, 정말, 연기, 가, 이것, 을, 어떻, 게, 보, ㄴ담, ㅎㅎㅎ, 손,...","[연기, 이것, 손, 손예진, 예진, 김, 삼류]","[와, 정말, 연기, 가, 이, 걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉...","[정말, 연기, 걸, 손예진, 김남길, 흉내내, 삼류]"
...,...,...,...,...,...,...,...,...,...,...
2995,영화내내 다섯살짜리 애기같은 스파이더맨에 끝까지 뺑이만 치는 닥터 스트레인지모든 스...,1,spiderman,영화내내 다섯살짜리 애기같은 스파이더맨에 끝까지 뺑이만 치는 닥터 스트레인지모든 스...,"[영화내내, 다섯살짜리, 애기같은, 스파이더맨에, 끝까지, 뺑이만, 치는, 닥터, ...","[영화내내, 다섯살짜리, 애기같은, 스파이더맨에, 끝까지, 뺑이만, 치는, 닥터, ...","[영화, 내내, 다섯, 살, 짜리, 애기, 같, 은, 스파이, 더, 맨, 에, 끝,...","[영화, 다섯, 다섯살, 살, 애기, 스파이, 스파이더맨, 더, 맨, 끝, 닥터, ...","[영화, 내내, 다섯, 살, 짜리, 애기, 같은, 스파이더맨, 에, 끝, 까지, 뺑...","[영화, 내내, 살, 애기, 스파이더맨, 끝, 뺑이, 닥터, 스, 트레인, 지모, ..."
2996,소니의 스파이더맨들과의 콜라보가 될 줄이야...,8,spiderman,소니의 스파이더맨들과의 콜라보가 될 줄이야,"[소니의, 스파이더맨들과의, 콜라보가, 될, 줄이야]","[소니의, 스파이더맨들과의, 콜라보가, 될, 줄이야]","[소니, 의, 스파이, 더, 맨들, ㄹ, 과의, 콜라, 보가, 되, ㄹ, 줄, 이, 야]","[소니, 스파이, 과의, 콜라, 콜라보가, 보가, 줄]","[소니, 의, 스파이더맨, 들, 과의, 콜라보, 가, 될, 줄, 이야]","[소니, 스파이더맨, 콜라보, 줄]"
2997,bad bad bad bad,2,batman,bad bad bad bad,"[bad, bad, bad, bad]","[bad, bad, bad, bad]","[bad, bad, bad, bad]",[],"[bad, bad, bad, bad]",[]
2998,bad bad bad bad,2,batman,bad bad bad bad,"[bad, bad, bad, bad]","[bad, bad, bad, bad]","[bad, bad, bad, bad]",[],"[bad, bad, bad, bad]",[]


(잠정적 결론)kkma와 okt 비교: okt가 나음

#벡터라이즈

In [ ]:
#라이브러리
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

In [ ]:
# CountVectorizer를 변수에 저장합니다.
vect = CountVectorizer()

In [ ]:
df.head(5)

,review_text,review_star,movie,clean_text,tokens,tokens_wo_sw,lem_kkma_morph,lem_kkma_noun,lem_okt_morph,lem_okt_noun
0,영화보면서 댓글남기는중... 최근 10년 동안본영화중 역대급으로 재미없음... 스토...,1,pirate,영화보면서 댓글남기는중 최근 10년 동안본영화중 역대급으로 재미없음 스토리 연기 특...,"[영화보면서, 댓글남기는중, 최근, 10년, 동안본영화중, 역대급으로, 재미없음, ...","[영화보면서, 댓글남기는중, 최근, 10년, 동안본영화중, 역대급으로, 재미없음, ...","[영화, 보, 면서, 댓, 글, 남기, 는, 중, 최근, 10, 년, 동안, 본, ...","[영화, 댓, 댓글, 글, 중, 최근, 10, 10년, 년, 동안, 동안본영화중, ...","[영화, 보면서, 댓글, 남기는, 중, 최근, 10년, 동안, 본, 영화, 중, 역...","[영화, 댓글, 중, 최근, 동안, 영화, 역대, 스토리, 연기, 특수, 효과, 등..."
1,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,10,batman,요즘 정치시즌에도 뭔가 잘맞는낌 강추합니다,"[요즘, 정치시즌에도, 뭔가, 잘맞는낌, 강추합니다]","[요즘, 정치시즌에도, 뭔가, 잘맞는낌, 강추합니다]","[요즘, 정치, 시즌, 에, 도, 뭔, 가, 잘, 맞, 는, 끼, ㅁ, 강추, 하,...","[요즘, 정치, 정치시즌, 시즌, 강추]","[요즘, 정치, 시즌, 에도, 뭔가, 잘맞는, 낌, 강, 추합니다]","[요즘, 정치, 시즌, 뭔가, 낌, 강]"
2,스토리 전개가 쥑입니다요..^^,10,spiderman,스토리 전개가 쥑입니다요 ^^,"[스토리, 전개가, 쥑입니다요, ^^]","[스토리, 전개가, 쥑입니다요, ^^]","[스토리, 전개, 가, 쥑이, ㅂ니다요, ^^]","[스토리, 전개]","[스토리, 전개, 가, 쥑, 입니다요, ^^]","[스토리, 전개, 쥑]"
3,bad bad bad bad,2,batman,bad bad bad bad,"[bad, bad, bad, bad]","[bad, bad, bad, bad]","[bad, bad, bad, bad]",[],"[bad, bad, bad, bad]",[]
4,와.. 정말 연기가.. 이걸 어떻게 본담 ㅎㅎㅎ 손예진 김남길 흉내내는 삼류..,1,pirate,와 정말 연기가 이걸 어떻게 본담 ㅎㅎㅎ 손예진 김남길 흉내내는 삼류,"[와, 정말, 연기가, 이걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉내내는,...","[정말, 연기가, 이걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉내내는, 삼류]","[오, 아, 정말, 연기, 가, 이것, 을, 어떻, 게, 보, ㄴ담, ㅎㅎㅎ, 손,...","[연기, 이것, 손, 손예진, 예진, 김, 삼류]","[와, 정말, 연기, 가, 이, 걸, 어떻게, 본담, ㅎㅎㅎ, 손예진, 김남길, 흉...","[정말, 연기, 걸, 손예진, 김남길, 흉내내, 삼류]"


In [ ]:
# 어휘 사전을 생성합니다.
count_vect = CountVectorizer(stop_words='english', max_features=100)

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_count = count_vect.fit_transform(df['clean_text'])

dtm_count = pd.DataFrame(dtm_count.todense(), columns=count_vect.get_feature_names())
dtm_count

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,100만도,bad,가장,감동적인,너무,로버트,만든다,베트맨,사람은,스토리,...,한효주,한효주배우님,한효주씨는,한효주와,해야하나,했다면,현실은,호들갑은,혼잣말로,홀랜드
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2997,0,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2998,0,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#tfidfVectorizer

In [ ]:
tfidf_vect = TfidfVectorizer(stop_words='english', max_features=100)

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_tfidf = tfidf_vect.fit_transform(df['clean_text'])

dtm_tfidf = pd.DataFrame(dtm_tfidf.todense(), columns=tfidf_vect.get_feature_names())
dtm_tfidf

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,100만도,bad,가장,감동적인,너무,로버트,만든다,베트맨,사람은,스토리,...,한효주,한효주배우님,한효주씨는,한효주와,해야하나,했다면,현실은,호들갑은,혼잣말로,홀랜드
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.243507,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.0,0.0,0.254415,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2996,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2997,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2998,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#유사도

In [ ]:
from sklearn.neighbors import NearestNeighbors

# dtm을 사용히 NN 모델을 학습시킵니다. (디폴트)최근접 5 이웃.
nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm_tfidf)

NearestNeighbors(algorithm='kd_tree')

In [ ]:
nn.kneighbors([dtm_tfidf.iloc[30]])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  "X does not have valid feature names, but"


(array([[0., 0., 0., 0., 0.]]), array([[ 91,  27,  30,  54, 100]]))

In [ ]:
nn.kneighbors([dtm_tfidf.iloc[18]])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  "X does not have valid feature names, but"


(array([[0., 0., 0., 0., 0.]]), array([[ 4, 71, 18, 13, 36]]))

In [ ]:
print(df['clean_text'][11][:300])


베트맨 시리즈는 처음이지만 로버트 패튼슨 버전의 베트맨 극호! 정보없이 봤는데 로버트 패튼슨 나와서 깜짝놀라고 너무 섹시해서 또 깜놀!! 세 시간이 길게 느껴지지 않았음


In [ ]:
print(df['clean_text'][14][:300])

영화내내 다섯살짜리 애기같은 스파이더맨에 끝까지 뺑이만 치는 닥터 스트레인지모든 스파이더맨 시리즈 중 가장 억지스러운 선함에 영화가 끝나고 악해지고 싶은 마음이 들 정도임
